<a href="https://colab.research.google.com/github/seungbinahn/START_AI/blob/master/02_Machine_Learning%E2%80%8E/07_2_%EC%86%90%EC%8B%A4%ED%95%A8%EC%88%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 딥러닝
* 지금까지는 가중치, 매개변수 값을 수작업으로 설정
* 모델학습 : 훈련 데이터로부터 가중치 매개변수의 최적값을 자동으로 획득
* 특정 지표를 활용하여 최적값 탐색

## 손실 함수
* Loss Function
: Neural Network의 최적 weight를 찾기 위한 지표
* 일반적으로 사용되는 손실함수
* MSE(Mean Squared Eroor) : 평균 제곱 오차
* GEE(Cross Entropy Error) : 교차 엔트로피 오차


### MSE
* 가장 많이 사용하는 Loss Function
* E = sum((Yk-Tk)^2) / 2
* Yk : 신경망의 출력(예측값), Tk: 정답 레이블

* 각 Output Node의 Prediction과 label 간의 차를 제곱한 후 총합을 더함

In [ ]:
import numpy as np

def mean_squared_error(y, t):
    return 0.5 * np.sum((y-t)**2)

In [ ]:
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
# 2일 확률이 가장 높다고 추정
t = [0,0,1,0,0,0,0,0,0,0]

mean_squared_error(np.array(y),np.array(t))

0.09750000000000003

In [ ]:
# 확률 분포를 구하고 (7이 가장 높음)
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
# 그런데 2일 확률이 가장 높다고 추정
t = [0,0,1,0,0,0,0,0,0,0]

# E 증가
mean_squared_error(np.array(y),np.array(t))

0.5975

### CEE
* Tk : 정답 레이블
* Yk : Output Node의 출력
* Tk 는 One-Hot Encoding이기 때문에 정답일 경우만 Tk값이 1
* 이때의 log Yk를 게산한 값이 CEE의 결과값
(정답일 떄의 출력이 전체 값 결정)

E = - sum(Tk log Yk)
* 정답에 해당하는 Output이 1일 때 0
* 정답에 해당하는 output이 작을수록 값이 점점 커짐
* 로그 그래프의 모양을 고려하면 됨

In [ ]:
import numpy as np

def cross_entropy_error(y, t):
    """
    delta는 로그 함수에 0이 들어가면 계산이 중지되므로 
    작은 값을 더해 이를 방지해줌
    """
    delta = 1e-7
    return -np.sum(t*np.log(y+delta))

In [ ]:
# 2일 확률이 가장 높은 데이터
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]

# 2일 확률이 가장 높다고 추정
t = [0,0,1,0,0,0,0,0,0,0] # one-hot

cross_entropy_error(np.array(y),np.array(t))

0.510825457099338

In [ ]:
# 7일 확률이 가장 높은 데이터
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]

# 2일 확률이 가장 높다고 추정
t = [0,0,1,0,0,0,0,0,0,0] 

# 이전 결과에 비하여 E 증가
cross_entropy_error(np.array(y),np.array(t))

2.302584092994546

## Batch 학습
* 모든 훈련 데이터를 대상으로 손실함수를 구하고, 그 값을 최대한 줄이는 것이 이상적
* 현실적으로 모든 데이터를 대상으로 손실 함수를 계산하기는 어려움
* 데이터의 일부를 추려 전체의 근사치로 이용할 수 있음
* 미니배치 : 훈련 데이터로부터 일부를 무작위로 뽑아 학습 수행

In [ ]:
# 데이터가 하나인 경우와 배치로 묶여 입력될 경우 모두 처리
def cross_batch_entropy_error(y, t):
    delta = 1e-7
    if y.ndim == 1: # 데이터가 1개인 경우 고려(차원 변경하여 평균 계산)
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
    batch_size = y.shape[0] # 행수를 보면 배치의 크기를 알 수 있음
    return -np.sum(t*np.log(y+delta)) / batch_size # 배치 크기로 나누어 정규화

In [ ]:
# 2일 확률이 가장 높은 데이터
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]

# 2일 확률이 가장 높다고 추정
t = [0,0,1,0,0,0,0,0,0,0] # one-hot

cross_batch_entropy_error(np.array(y),np.array(t))

0.510825457099338

In [ ]:
# 7일 확률이 가장 높은 데이터
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]

# 2일 확률이 가장 높다고 추정
t = [0,0,1,0,0,0,0,0,0,0] 

# 이전 결과에 비하여 E 증가
cross_batch_entropy_error(np.array(y),np.array(t))

2.302584092994546